In [1]:
import tensorflow as tf
tf.InteractiveSession()

In [2]:
x = tf.constant([[0.25, 0.4, 0.3, 0.05], 
                 [0.005, 0.005, 0.44, 0.55],
                 [0.5, 0.25, 0.15, 0.1],
                 [0.3, 0.2, 0.4, 0.1],
                 [0.7, 0.05, 0.05, 0.2],
                 [0.6, 0.2, 0.2, 0.1]])
batch_size = 2 
beam_size = 3
num_classes = 4

In [3]:
x.eval()

array([[ 0.25      ,  0.40000001,  0.30000001,  0.05      ],
       [ 0.005     ,  0.005     ,  0.44      ,  0.55000001],
       [ 0.5       ,  0.25      ,  0.15000001,  0.1       ],
       [ 0.30000001,  0.2       ,  0.40000001,  0.1       ],
       [ 0.69999999,  0.05      ,  0.05      ,  0.2       ],
       [ 0.60000002,  0.2       ,  0.2       ,  0.1       ]], dtype=float32)

In [4]:
logprobs_batched = tf.reshape(x, [batch_size, beam_size, num_classes])

In [5]:
logprobs_batched.eval()

array([[[ 0.25      ,  0.40000001,  0.30000001,  0.05      ],
        [ 0.005     ,  0.005     ,  0.44      ,  0.55000001],
        [ 0.5       ,  0.25      ,  0.15000001,  0.1       ]],

       [[ 0.30000001,  0.2       ,  0.40000001,  0.1       ],
        [ 0.69999999,  0.05      ,  0.05      ,  0.2       ],
        [ 0.60000002,  0.2       ,  0.2       ,  0.1       ]]], dtype=float32)

In [6]:
logprobs_batched[0].eval()

array([[ 0.25      ,  0.40000001,  0.30000001,  0.05      ],
       [ 0.005     ,  0.005     ,  0.44      ,  0.55000001],
       [ 0.5       ,  0.25      ,  0.15000001,  0.1       ]], dtype=float32)

In [7]:
tf.reshape(logprobs_batched, [-1, beam_size * num_classes]).eval()

array([[ 0.25      ,  0.40000001,  0.30000001,  0.05      ,  0.005     ,
         0.005     ,  0.44      ,  0.55000001,  0.5       ,  0.25      ,
         0.15000001,  0.1       ],
       [ 0.30000001,  0.2       ,  0.40000001,  0.1       ,  0.69999999,
         0.05      ,  0.05      ,  0.2       ,  0.60000002,  0.2       ,
         0.2       ,  0.1       ]], dtype=float32)

In [8]:
past_logprobs, indices = tf.nn.top_k(tf.reshape(logprobs_batched, [-1, beam_size * num_classes]), beam_size)

In [9]:
past_logprobs.eval()

array([[ 0.55000001,  0.5       ,  0.44      ],
       [ 0.69999999,  0.60000002,  0.40000001]], dtype=float32)

In [10]:
indices.eval()

array([[7, 8, 6],
       [4, 8, 2]], dtype=int32)

In [11]:
symbols = indices % num_classes

In [12]:
symbols.eval()

array([[3, 0, 2],
       [0, 0, 2]], dtype=int32)

In [13]:
parent_refs = indices // num_classes

In [14]:
parent_refs.eval()

array([[1, 2, 1],
       [1, 2, 0]], dtype=int32)

In [15]:
parent_refs_offsets = tf.reshape(
          (tf.range(batch_size * beam_size) // beam_size) * beam_size,
          [batch_size, beam_size]
      )

In [16]:
parent_refs_offsets.eval()

array([[0, 0, 0],
       [3, 3, 3]], dtype=int32)

In [17]:
past_symbols = tf.expand_dims(symbols, 2)

In [18]:
past_symbols.eval()

array([[[3],
        [0],
        [2]],

       [[0],
        [0],
        [2]]], dtype=int32)

In [19]:
past_symbols_batch_major = tf.reshape(past_symbols, [-1, 1])

In [20]:
past_symbols_batch_major.eval()

array([[3],
       [0],
       [2],
       [0],
       [0],
       [2]], dtype=int32)

In [21]:
(parent_refs + parent_refs_offsets).eval()

array([[1, 2, 1],
       [4, 5, 3]], dtype=int32)

In [22]:
 beam_past_symbols = tf.gather(past_symbols_batch_major, #batch-major
                               parent_refs + parent_refs_offsets)

In [23]:
beam_past_symbols.eval()

array([[[0],
        [2],
        [0]],

       [[0],
        [2],
        [0]]], dtype=int32)

In [24]:
beam_past_symbols.eval().shape

(2, 3, 1)

In [25]:
x.eval()

array([[ 0.25      ,  0.40000001,  0.30000001,  0.05      ],
       [ 0.005     ,  0.005     ,  0.44      ,  0.55000001],
       [ 0.5       ,  0.25      ,  0.15000001,  0.1       ],
       [ 0.30000001,  0.2       ,  0.40000001,  0.1       ],
       [ 0.69999999,  0.05      ,  0.05      ,  0.2       ],
       [ 0.60000002,  0.2       ,  0.2       ,  0.1       ]], dtype=float32)

In [26]:
x[:,0].eval()

array([ 0.25      ,  0.005     ,  0.5       ,  0.30000001,  0.69999999,
        0.60000002], dtype=float32)

In [27]:
x[:,0:1].eval()

array([[ 0.25      ],
       [ 0.005     ],
       [ 0.5       ],
       [ 0.30000001],
       [ 0.69999999],
       [ 0.60000002]], dtype=float32)

In [28]:
past_symbols.eval()

array([[[3],
        [0],
        [2]],

       [[0],
        [0],
        [2]]], dtype=int32)

In [29]:
past_symbols[:, 1, :].eval()

array([[0],
       [0]], dtype=int32)

In [30]:
past_symbols.eval().shape

(2, 3, 1)

In [31]:
tf.tile(past_symbols, [1, 1, 3]).eval()

array([[[3, 3, 3],
        [0, 0, 0],
        [2, 2, 2]],

       [[0, 0, 0],
        [0, 0, 0],
        [2, 2, 2]]], dtype=int32)

In [32]:
tf.tile(past_symbols, [1, 1, 3])[:,1,:].eval()

array([[0, 0, 0],
       [0, 0, 0]], dtype=int32)